In [149]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import urllib.parse

In [150]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [151]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [152]:
%%capture pwd
!pwd

In [153]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
uid=urllib.parse.quote(uid)
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('how-did-countries-experiencing-internal-conflict-cope-with-the-pandemic',
 300,
 500)

# Fig 1

## a

In [154]:
df = pd.read_excel("raw/fig_1_data_excel.xlsx").dropna()

In [155]:
f = "fig1a_d_rates"
f1a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1a = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,code,continent,country,internal_conf_icrg_g_21_20 (in %),log_totaldeathspermil2021_2020,internal_conf_icrg_21minus20,general_internalconf_g
2,AGO,Africa,Angola,0.92,3.954316,0.04,0
3,ALB,Europe,Albania,-7.95,7.020870,-0.29,1
5,ARE,Asia,United Arab Emirates,-12.79,5.378006,-0.45,1
6,ARG,South America,Argentina,7.45,7.851338,0.29,0
7,ARM,Asia,Armenia,3.06,7.895760,0.13,0


In [156]:
base = alt.Chart(f1a).encode(
    x=alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="COVID-19 death rates per million at the end of 2021",
            titleY=-15,
            # titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
        scale=alt.Scale(
            type='log'
        )
    )
).transform_calculate(x='exp(datum.log_totaldeathspermil2021_2020)').transform_filter('datum.general_internalconf_g==1')
points = base.mark_point(fill=colors["eco-light-blue"],color=colors["eco-mid-blue"]).encode(
    y=alt.Y(
        "internal_conf_icrg_21minus20:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Absolute change in internal conflict between 2020 and 2021",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        # scale=alt.Scale(domain=[-1.4, 1.5]),
    )
)
labels=points.mark_text(color=colors["eco-mid-blue"],fontSize=9).encode(text='code:N')
fit = base.transform_regression(
        "log_totaldeathspermil2021_2020", "internal_conf_icrg_21minus20"
    ).transform_calculate(x2='exp(datum.log_totaldeathspermil2021_2020)')\
.mark_line(opacity=0.5).encode(x='x2:Q',y='internal_conf_icrg_21minus20:Q')
layer1 = (
    ((fit+labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

WARN Loading failed https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/articles/how-did-countries-experiencing-internal-conflict-cope-with-the-pandemic/data/fig1a_d_rates.csv Error: 404Not Found
    at Object.error (/opt/conda/lib/node_modules/vega-cli/node_modules/vega-util/build/vega-util.js:39:11)
    at Object.http (/opt/conda/lib/node_modules/vega-cli/node_modules/vega-loader/build/vega-loader.node.js:173:38)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async View.request (/opt/conda/lib/node_modules/vega-cli/node_modules/vega-dataflow/build/vega-dataflow.js:946:14)
    at async View.preload (/opt/conda/lib/node_modules/vega-cli/node_modules/vega-dataflow/build/vega-dataflow.js:971:17)
WARN Skipping regression with more parameters than data points.
WARN Infinite extent for field "x2": [Infinity, -Infinity]
WARN Infinite extent for field "x": [Infinity, -Infinity]
WARN Infinite extent for field "internal_conf_ic

alt.LayerChart(...)

# Fig 1

## b

In [157]:
f = "fig1b_d_change_perc"
f1b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1b = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,code,continent,country,internal_conf_icrg_g_21_20 (in %),log_totaldeathspermil2021_2020,internal_conf_icrg_21minus20,general_internalconf_g
2,AGO,Africa,Angola,0.92,3.954316,0.04,0
3,ALB,Europe,Albania,-7.95,7.020870,-0.29,1
5,ARE,Asia,United Arab Emirates,-12.79,5.378006,-0.45,1
6,ARG,South America,Argentina,7.45,7.851338,0.29,0
7,ARM,Asia,Armenia,3.06,7.895760,0.13,0


In [158]:
base = alt.Chart(f1b).encode(
    x=alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="COVID-19 death rates per million at the end of 2021",
            titleY=-15,
            # titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
        scale=alt.Scale(
            type='log'
        )
    )
).transform_calculate(x='exp(datum.log_totaldeathspermil2021_2020)').transform_filter('datum.general_internalconf_g==1')
points = base.mark_point(fill=colors["eco-light-blue"],color=colors["eco-mid-blue"]).encode(
    y=alt.Y(
        "internal_conf_icrg_g_21_20 (in %):Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Relative change in internal conflict between 2020 and 2021 (%)",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[-25, 30]),
    )
)
labels=points.mark_text(color=colors["eco-mid-blue"],fontSize=9).encode(text='code:N')
fit = base.transform_regression(
        "log_totaldeathspermil2021_2020", "internal_conf_icrg_g_21_20 (in %)"
    ).transform_calculate(x2='exp(datum.log_totaldeathspermil2021_2020)')\
.mark_line(opacity=0.5).encode(x='x2:Q',y='internal_conf_icrg_g_21_20 (in %):Q')
layer1 = (
    ((fit+labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

WARN Loading failed https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/articles/how-did-countries-experiencing-internal-conflict-cope-with-the-pandemic/data/fig1b_d_change_perc.csv Error: 404Not Found
    at Object.error (/opt/conda/lib/node_modules/vega-cli/node_modules/vega-util/build/vega-util.js:39:11)
    at Object.http (/opt/conda/lib/node_modules/vega-cli/node_modules/vega-loader/build/vega-loader.node.js:173:38)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async View.request (/opt/conda/lib/node_modules/vega-cli/node_modules/vega-dataflow/build/vega-dataflow.js:946:14)
    at async View.preload (/opt/conda/lib/node_modules/vega-cli/node_modules/vega-dataflow/build/vega-dataflow.js:971:17)
WARN Skipping regression with more parameters than data points.
WARN Infinite extent for field "x2": [Infinity, -Infinity]
WARN Infinite extent for field "x": [Infinity, -Infinity]
WARN Log scale domain includes zero: [Infi

alt.LayerChart(...)

# Fig 2

In [159]:
df = pd.read_excel("raw/fig_2_data_excel.xlsx").dropna()
df

,code,continent,country,civildisorder_icrg2020,macrofin_gdp,log_gdppc_ppp2019,log_totaldeathspermil2021_2020,civildisorder_g_icrg21_20,inf_2020_19_ave,dummy_cont1,dummy_cont2,dummy_cont3,dummy_cont4,dummy_cont6,general_internalconf_g
3,ALB,Europe,Albania,2.67,4.000000,9.521978,7.020870,-10.861422,1.515,0.0,0.0,1.0,0.0,0.0,1
5,ARE,Asia,United Arab Emirates,2.08,29.969999,11.131133,5.378006,-3.846150,-2.005,0.0,1.0,0.0,0.0,0.0,1
7,ARM,Asia,Armenia,2.58,0.000000,9.521773,7.895760,5.038764,1.325,0.0,1.0,0.0,0.0,0.0,0
9,AUS,Oceania,Australia,1.88,16.660000,10.805864,4.470037,10.638294,1.230,0.0,0.0,0.0,0.0,0.0,1
10,AUT,Europe,Austria,2.00,18.639999,10.930131,7.325557,6.500006,1.455,0.0,0.0,1.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,URY,South America,Uruguay,2.04,5.260000,10.044670,7.478944,1.960783,8.820,0.0,0.0,0.0,0.0,1.0,1
197,USA,North America,United States,2.13,14.000000,11.045014,7.817980,-4.225359,1.520,0.0,0.0,0.0,1.0,0.0,1
202,VNM,Asia,Vietnam,2.00,10.120000,8.992332,5.799032,3.999996,3.010,0.0,1.0,0.0,0.0,0.0,1
207,ZAF,Africa,South Africa,2.42,0.630000,9.525881,7.325162,4.958673,3.670,1.0,0.0,0.0,0.0,0.0,1


In [160]:
f = "fig2_effects"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,code,continent,country,civildisorder_icrg2020,macrofin_gdp,log_gdppc_ppp2019,log_totaldeathspermil2021_2020,civildisorder_g_icrg21_20,inf_2020_19_ave,dummy_cont1,dummy_cont2,dummy_cont3,dummy_cont4,dummy_cont6,general_internalconf_g
3,ALB,Europe,Albania,2.67,4.000000,9.521978,7.020870,-10.861422,1.515,0.0,0.0,1.0,0.0,0.0,1
5,ARE,Asia,United Arab Emirates,2.08,29.969999,11.131133,5.378006,-3.846150,-2.005,0.0,1.0,0.0,0.0,0.0,1
7,ARM,Asia,Armenia,2.58,0.000000,9.521773,7.895760,5.038764,1.325,0.0,1.0,0.0,0.0,0.0,0
9,AUS,Oceania,Australia,1.88,16.660000,10.805864,4.470037,10.638294,1.230,0.0,0.0,0.0,0.0,0.0,1
10,AUT,Europe,Austria,2.00,18.639999,10.930131,7.325557,6.500006,1.455,0.0,0.0,1.0,0.0,0.0,1
